In [1]:
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))

2021-11-19 06:43:12.861383: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


2021-11-19 06:43:22.061635: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-19 06:43:22.096613: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-11-19 06:43:22.398705: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-19 06:43:22.399560: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.545GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.6

In [2]:
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

2021-11-19 06:43:28.877963: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-11-19 06:43:28.878031: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-19 06:43:28.878436: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.545GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2021-11-19 06:43:28.878478: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-19 06:43:28.878883: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:04:00

In [3]:
import numpy as np
import pandas as pd

In [4]:
# from keras.models import Sequential
# from keras.layers import Dense
# from keras.layers import LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout

In [5]:
# 顯示所有列
pd.set_option('display.max_columns', None)

In [6]:
# 讀前處理過後的資料
# train = pd.read_csv('Processed_tbrain_training.csv')

# 讀合併完消費次數的看看會不會比較快
train = pd.read_csv('Processed_tbrain_training.csv')
print(train.shape)
print(train.info())
train.head(5)

(32970085, 53)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32970085 entries, 0 to 32970084
Data columns (total 53 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   dt                        int64  
 1   chid                      int64  
 2   shop_tag                  int64  
 3   txn_cnt                   int64  
 4   txn_amt                   float64
 5   domestic_offline_cnt      int64  
 6   domestic_online_cnt       int64  
 7   overseas_offline_cnt      int64  
 8   overseas_online_cnt       int64  
 9   domestic_offline_amt_pct  float64
 10  domestic_online_amt_pct   float64
 11  overseas_offline_amt_pct  float64
 12  overseas_online_amt_pct   float64
 13  card_1_txn_cnt            int64  
 14  card_2_txn_cnt            int64  
 15  card_3_txn_cnt            int64  
 16  card_4_txn_cnt            int64  
 17  card_5_txn_cnt            int64  
 18  card_6_txn_cnt            int64  
 19  card_7_txn_cnt            int64  
 20  card_8_

,dt,chid,shop_tag,txn_cnt,txn_amt,domestic_offline_cnt,domestic_online_cnt,overseas_offline_cnt,overseas_online_cnt,domestic_offline_amt_pct,domestic_online_amt_pct,overseas_offline_amt_pct,overseas_online_amt_pct,card_1_txn_cnt,card_2_txn_cnt,card_3_txn_cnt,card_4_txn_cnt,card_5_txn_cnt,card_6_txn_cnt,card_7_txn_cnt,card_8_txn_cnt,card_9_txn_cnt,card_10_txn_cnt,card_11_txn_cnt,card_12_txn_cnt,card_13_txn_cnt,card_14_txn_cnt,card_other_txn_cnt,card_1_txn_amt_pct,card_2_txn_amt_pct,card_3_txn_amt_pct,card_4_txn_amt_pct,card_5_txn_amt_pct,card_6_txn_amt_pct,card_7_txn_amt_pct,card_8_txn_amt_pct,card_9_txn_amt_pct,card_10_txn_amt_pct,card_11_txn_amt_pct,card_12_txn_amt_pct,card_13_txn_amt_pct,card_14_txn_amt_pct,card_other_txn_amt_pct,masts,educd,trdtp,naty,poscd,cuorg,slam,gender_code,age,primary_card
0,1,10321418,45,3,5.830816e-112,3,0,0,0,1.00,0.00,0.0,0.0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,5.0,1.0,99.0,30.0,1.394080e-213,1.0,4.0,1
1,1,10414574,15,2,1.596363e-111,2,0,0,0,1.00,0.00,0.0,0.0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,15.0,1.0,2.0,30.0,1.944332e-213,1.0,3.0,1
2,1,10134567,48,2,3.541844e-111,0,2,0,0,0.00,1.00,0.0,0.0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,11.0,1.0,3.0,30.0,1.648096e-213,0.0,4.0,1
3,1,10001003,48,9,2.671485e-111,7,2,0,0,0.75,0.25,0.0,0.0,0,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,11.0,1.0,2.0,30.0,8.190756e-214,0.0,3.0,1
4,1,10267183,2,1,3.266644e-111,1,0,0,0,1.00,0.00,0.0,0.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,6.0,15.0,1.0,99.0,30.0,8.190756e-214,0.0,6.0,0


In [7]:
# 特徵選擇
train.columns

Index(['dt', 'chid', 'shop_tag', 'txn_cnt', 'txn_amt', 'domestic_offline_cnt',
       'domestic_online_cnt', 'overseas_offline_cnt', 'overseas_online_cnt',
       'domestic_offline_amt_pct', 'domestic_online_amt_pct',
       'overseas_offline_amt_pct', 'overseas_online_amt_pct', 'card_1_txn_cnt',
       'card_2_txn_cnt', 'card_3_txn_cnt', 'card_4_txn_cnt', 'card_5_txn_cnt',
       'card_6_txn_cnt', 'card_7_txn_cnt', 'card_8_txn_cnt', 'card_9_txn_cnt',
       'card_10_txn_cnt', 'card_11_txn_cnt', 'card_12_txn_cnt',
       'card_13_txn_cnt', 'card_14_txn_cnt', 'card_other_txn_cnt',
       'card_1_txn_amt_pct', 'card_2_txn_amt_pct', 'card_3_txn_amt_pct',
       'card_4_txn_amt_pct', 'card_5_txn_amt_pct', 'card_6_txn_amt_pct',
       'card_7_txn_amt_pct', 'card_8_txn_amt_pct', 'card_9_txn_amt_pct',
       'card_10_txn_amt_pct', 'card_11_txn_amt_pct', 'card_12_txn_amt_pct',
       'card_13_txn_amt_pct', 'card_14_txn_amt_pct', 'card_other_txn_amt_pct',
       'masts', 'educd', 'trdtp', 'naty

In [8]:
# # 之前重要度最高的紀錄
# # txn_amt、slam兩欄小數點似乎過多，要再處理，這邊先直接砍掉
# SelectedFeatures = ['dt', 'chid',  'txn_cnt', 'domestic_offline_cnt',
#        'domestic_online_cnt', 'overseas_offline_cnt', 'overseas_online_cnt',
#        'domestic_offline_amt_pct', 'domestic_online_amt_pct',
#        'overseas_offline_amt_pct', 'overseas_online_amt_pct', 
#        'masts', 'educd', 'trdtp', 'naty', 'poscd', 'cuorg', 
#        'gender_code', 'age']

In [9]:
# 這個爆爛 0.1957188951	
# # 改一下之前重要度最高的紀錄
# SelectedFeatures = ['dt', 'chid', 'shop_tag', 'txn_amt', 'txn_cnt', 'domestic_offline_cnt',
#        'domestic_online_cnt', 'overseas_offline_cnt', 'overseas_online_cnt',
#        'domestic_offline_amt_pct', 'domestic_online_amt_pct',
#        'overseas_offline_amt_pct', 'overseas_online_amt_pct', 
#        'masts', 'educd', 'trdtp', 'naty', 'poscd', 'cuorg', 'slam',
#        'gender_code', 'age', 'primary_card']

In [10]:
# SelectedFeatures = ['dt', 'chid', 'shop_tag', 
#        'masts', 'educd', 'trdtp', 'naty', 'poscd', 'cuorg', 'slam',
#        'gender_code', 'age', 'primary_card']

In [11]:
# 多一個txn_amt消費金額來訓練
SelectedFeatures = ['dt', 'chid', 'shop_tag', 'txn_amt',
       'masts', 'educd', 'trdtp', 'naty', 'poscd', 'cuorg', 'slam',
       'gender_code', 'age', 'primary_card']

In [12]:
train = train[SelectedFeatures]
train.head(5)

,dt,chid,shop_tag,txn_amt,masts,educd,trdtp,naty,poscd,cuorg,slam,gender_code,age,primary_card
0,1,10321418,45,5.830816e-112,1.0,4.0,5.0,1.0,99.0,30.0,1.394080e-213,1.0,4.0,1
1,1,10414574,15,1.596363e-111,2.0,2.0,15.0,1.0,2.0,30.0,1.944332e-213,1.0,3.0,1
2,1,10134567,48,3.541844e-111,1.0,3.0,11.0,1.0,3.0,30.0,1.648096e-213,0.0,4.0,1
3,1,10001003,48,2.671485e-111,2.0,3.0,11.0,1.0,2.0,30.0,8.190756e-214,0.0,3.0,1
4,1,10267183,2,3.266644e-111,2.0,6.0,15.0,1.0,99.0,30.0,8.190756e-214,0.0,6.0,0


In [13]:
# 把每位顧客每個月消費金額最多的資料留下，不是消費金額最多的不留
train = train.iloc[train.groupby(['chid','dt']).apply(lambda x: x['txn_amt'].idxmax())]

In [14]:
train

,dt,chid,shop_tag,txn_amt,masts,educd,trdtp,naty,poscd,cuorg,slam,gender_code,age,primary_card
460091,1,10000000,10,1.684984e-111,2.0,2.0,2.0,1.0,3.0,30.0,1.133767e-213,1.0,4.0,1
1441185,2,10000000,46,2.805564e-111,2.0,2.0,2.0,1.0,3.0,30.0,1.133767e-213,1.0,4.0,1
2476546,3,10000000,10,1.719186e-111,2.0,2.0,2.0,1.0,3.0,30.0,1.133767e-213,1.0,4.0,1
3546273,4,10000000,10,1.220472e-111,2.0,2.0,2.0,1.0,3.0,30.0,1.133767e-213,1.0,4.0,1
4656553,5,10000000,10,2.045684e-111,2.0,2.0,2.0,1.0,3.0,30.0,1.133767e-213,1.0,4.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22529048,18,10499999,22,8.656715e-112,1.0,5.0,15.0,1.0,2.0,30.0,1.394080e-213,0.0,5.0,1
24083656,19,10499999,22,1.143671e-111,1.0,5.0,15.0,1.0,2.0,30.0,1.394080e-213,0.0,5.0,1
25611344,20,10499999,45,1.087033e-111,1.0,5.0,15.0,1.0,2.0,30.0,1.394080e-213,0.0,5.0,1
28777944,22,10499999,22,6.324116e-112,1.0,5.0,15.0,1.0,2.0,30.0,1.394080e-213,0.0,5.0,1


In [20]:
train.loc[train['chid'] == 10128239]

,dt,chid,shop_tag,txn_amt,masts,educd,trdtp,naty,poscd,cuorg,slam,gender_code,age,primary_card
1547006,2,10128239,0,1.086396e-111,1.0,4.0,8.0,1.0,1.0,30.0,1.741756e-213,0.0,4.0,1
3659342,4,10128239,0,1.086396e-111,1.0,4.0,8.0,1.0,1.0,30.0,1.741756e-213,0.0,4.0,1
4778336,5,10128239,0,7.751448e-112,1.0,4.0,8.0,1.0,1.0,30.0,1.741756e-213,0.0,4.0,1
5957260,6,10128239,0,1.086396e-111,1.0,4.0,8.0,1.0,1.0,30.0,1.741756e-213,0.0,4.0,1
9759238,9,10128239,22,2.274302e-112,1.0,4.0,8.0,1.0,1.0,30.0,1.741756e-213,0.0,4.0,1
13963327,12,10128239,0,1.086396e-111,1.0,4.0,8.0,1.0,1.0,30.0,1.741756e-213,0.0,4.0,1
19651936,16,10128239,37,5.347319e-112,1.0,4.0,8.0,1.0,1.0,30.0,1.741756e-213,0.0,4.0,1
21215971,17,10128239,0,7.751448e-112,1.0,4.0,8.0,1.0,1.0,30.0,1.741756e-213,0.0,4.0,1
22803880,18,10128239,0,1.086396e-111,1.0,4.0,8.0,1.0,1.0,30.0,1.741756e-213,0.0,4.0,1
25881732,20,10128239,45,9.472215e-112,1.0,4.0,8.0,1.0,1.0,30.0,1.741756e-213,0.0,4.0,1


In [21]:
# 先取出每個顧客不重複最後一個月的值之後預測用
pre_train = train.drop_duplicates(subset=['chid'], keep='last')
pre_train

,dt,chid,shop_tag,txn_amt,masts,educd,trdtp,naty,poscd,cuorg,slam,gender_code,age,primary_card
32143063,24,10000000,10,1.753713e-111,2.0,2.0,2.0,1.0,3.0,30.0,1.133767e-213,1.0,4.0,1
32174692,24,10000001,21,1.537124e-111,2.0,4.0,11.0,1.0,3.0,30.0,8.670854e-214,0.0,5.0,1
28609193,22,10000002,22,5.347319e-112,2.0,4.0,13.0,1.0,99.0,30.0,1.129543e-213,0.0,3.0,1
27389011,21,10000003,37,1.205646e-111,2.0,6.0,15.0,1.0,99.0,30.0,1.129543e-213,0.0,6.0,1
21969528,18,10000004,31,9.472215e-112,2.0,5.0,15.0,1.0,99.0,30.0,6.369410e-214,1.0,2.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24142947,19,10499995,10,5.566171e-112,1.0,3.0,5.0,1.0,99.0,30.0,1.258009e-213,1.0,4.0,1
32555104,24,10499996,36,8.665268e-112,2.0,3.0,11.0,1.0,2.0,30.0,7.182541e-214,1.0,3.0,1
32633777,24,10499997,36,1.290124e-111,1.0,6.0,8.0,1.0,2.0,30.0,1.017454e-213,0.0,3.0,1
31618777,24,10499998,10,2.704133e-111,2.0,6.0,13.0,1.0,99.0,30.0,9.594880e-214,1.0,4.0,1


In [22]:
# 去除shop_tag
pre_train.drop(columns = ['shop_tag'], axis=1, inplace=True)

/mnt/sdb/home/hongjun/.conda/envs/python38/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [23]:
predict = pd.DataFrame(pd.read_csv('需預測的顧客名單及提交檔案範例.csv'))
predict = predict.drop(columns = ['top1','top2','top3'])
print(predict.shape)
print(predict.info())
predict.head(5)

(500000, 1)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 1 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   chid    500000 non-null  int64
dtypes: int64(1)
memory usage: 3.8 MB
None


,chid
0,10128239
1,10077943
2,10277876
3,10364842
4,10392717


In [24]:
# 按照要提交的檔案排序
pre_train = pre_train.set_index('chid', drop= False)
pre_train = pre_train.loc[predict.chid]
pre_train.head(5)

,dt,chid,txn_amt,masts,educd,trdtp,naty,poscd,cuorg,slam,gender_code,age,primary_card
chid,,,,,,,,,,,,,
10128239,24,10128239,1.580629e-111,1.0,4.0,8.0,1.0,1.0,30.0,1.741756e-213,0.0,4.0,1
10077943,24,10077943,1.086396e-111,1.0,5.0,12.0,1.0,2.0,30.0,7.594606e-214,0.0,5.0,1
10277876,24,10277876,1.808198e-111,1.0,5.0,6.0,1.0,1.0,30.0,7.077406e-214,0.0,3.0,1
10364842,24,10364842,1.766926e-111,2.0,5.0,7.0,1.0,99.0,30.0,1.048096e-213,1.0,4.0,1
10392717,24,10392717,3.243291e-111,2.0,3.0,5.0,1.0,2.0,30.0,3.248696e-214,1.0,2.0,1


In [25]:
pre_train.tail(5)

,dt,chid,txn_amt,masts,educd,trdtp,naty,poscd,cuorg,slam,gender_code,age,primary_card
chid,,,,,,,,,,,,,
10433239,24,10433239,6.887733e-111,2.0,3.0,12.0,1.0,1.0,30.0,1.237607e-213,1.0,3.0,1
10235119,24,10235119,2.154757e-111,2.0,3.0,17.0,1.0,1.0,30.0,1.133767e-213,1.0,4.0,1
10267595,24,10267595,1.867759e-111,2.0,3.0,8.0,1.0,2.0,30.0,1.648096e-213,1.0,3.0,1
10160606,24,10160606,7.751448e-112,2.0,4.0,6.0,1.0,99.0,30.0,1.017454e-213,0.0,5.0,1
10141759,24,10141759,2.710763e-111,2.0,6.0,15.0,1.0,99.0,30.0,1.217090e-213,0.0,2.0,0


In [26]:
# 把chid全部拿掉
pre_train.drop(columns = ['chid'], axis=1, inplace=True)
pre_train

,dt,txn_amt,masts,educd,trdtp,naty,poscd,cuorg,slam,gender_code,age,primary_card
chid,,,,,,,,,,,,
10128239,24,1.580629e-111,1.0,4.0,8.0,1.0,1.0,30.0,1.741756e-213,0.0,4.0,1
10077943,24,1.086396e-111,1.0,5.0,12.0,1.0,2.0,30.0,7.594606e-214,0.0,5.0,1
10277876,24,1.808198e-111,1.0,5.0,6.0,1.0,1.0,30.0,7.077406e-214,0.0,3.0,1
10364842,24,1.766926e-111,2.0,5.0,7.0,1.0,99.0,30.0,1.048096e-213,1.0,4.0,1
10392717,24,3.243291e-111,2.0,3.0,5.0,1.0,2.0,30.0,3.248696e-214,1.0,2.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
10433239,24,6.887733e-111,2.0,3.0,12.0,1.0,1.0,30.0,1.237607e-213,1.0,3.0,1
10235119,24,2.154757e-111,2.0,3.0,17.0,1.0,1.0,30.0,1.133767e-213,1.0,4.0,1
10267595,24,1.867759e-111,2.0,3.0,8.0,1.0,2.0,30.0,1.648096e-213,1.0,3.0,1


In [27]:
train.drop(columns = ['chid'], axis=1, inplace=True)
train

,dt,shop_tag,txn_amt,masts,educd,trdtp,naty,poscd,cuorg,slam,gender_code,age,primary_card
460091,1,10,1.684984e-111,2.0,2.0,2.0,1.0,3.0,30.0,1.133767e-213,1.0,4.0,1
1441185,2,46,2.805564e-111,2.0,2.0,2.0,1.0,3.0,30.0,1.133767e-213,1.0,4.0,1
2476546,3,10,1.719186e-111,2.0,2.0,2.0,1.0,3.0,30.0,1.133767e-213,1.0,4.0,1
3546273,4,10,1.220472e-111,2.0,2.0,2.0,1.0,3.0,30.0,1.133767e-213,1.0,4.0,1
4656553,5,10,2.045684e-111,2.0,2.0,2.0,1.0,3.0,30.0,1.133767e-213,1.0,4.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
22529048,18,22,8.656715e-112,1.0,5.0,15.0,1.0,2.0,30.0,1.394080e-213,0.0,5.0,1
24083656,19,22,1.143671e-111,1.0,5.0,15.0,1.0,2.0,30.0,1.394080e-213,0.0,5.0,1
25611344,20,45,1.087033e-111,1.0,5.0,15.0,1.0,2.0,30.0,1.394080e-213,0.0,5.0,1
28777944,22,22,6.324116e-112,1.0,5.0,15.0,1.0,2.0,30.0,1.394080e-213,0.0,5.0,1


In [28]:
# 看類別不平均的問題
train.shop_tag.value_counts()

37    1715329
10     763810
0      694846
2      669107
45     530365
36     485187
48     458509
15     424685
16     394608
19     273574
12     261142
39     183114
6      163980
46     156816
26     134084
17     131778
18     127780
34     122944
11     104130
41      95678
25      95316
35      92177
31      89076
22      86402
21      85278
13      78644
20      74072
43      56729
9       51076
33      42018
38      40032
40      27499
23      27255
5       26742
32      23337
3       23331
47      22174
8       19442
30      16598
1       15816
24      14525
7       10983
14       9998
44       9965
42       8491
28       4353
4        3786
27       1683
29         26
Name: shop_tag, dtype: int64

In [29]:
# 調類別權重
from sklearn.utils import class_weight

In [30]:
# 拆分測試訓練集
from sklearn.model_selection import train_test_split
train, test = train_test_split(train, test_size=0.2, random_state=42)

In [31]:
target_column_train = ['shop_tag']
predictors_train = list(set(list(train.columns))-set(target_column_train))

X_train = train[predictors_train].values
y_train = train[target_column_train].values

print(X_train.shape)
print(y_train.shape)

(7158632, 12)
(7158632, 1)


In [32]:
target_column_test = ['shop_tag'] 
predictors_test = list(set(list(test.columns))-set(target_column_test))

X_test = test[predictors_test].values
y_test = test[target_column_test].values

print(X_test.shape)
print(y_test.shape)

(1789658, 12)
(1789658, 1)


In [33]:
import gc
del train
del test
gc.collect()

0

In [34]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(7158632, 12)
(7158632, 1)
(1789658, 12)
(1789658, 1)


In [35]:
X_train1 = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [36]:
X_train1.shape

(7158632, 12, 1)

In [37]:
y_train1 = np.reshape(y_train, (y_train.shape[0], y_train.shape[1], 1))

In [38]:
y_train1.shape

(7158632, 1, 1)

In [39]:
X_test1 = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
X_test1.shape

(1789658, 12, 1)

In [40]:
y_test1 = np.reshape(y_test, (y_test.shape[0], y_test.shape[1], 1))
y_test1.shape

(1789658, 1, 1)

In [41]:
pre_train = pre_train.values
pre_train = np.reshape(pre_train, (pre_train.shape[0], pre_train.shape[1], 1))
pre_train.shape

(500000, 12, 1)

In [42]:
# X_train[..., None].shape

In [43]:
# activation = 'softmax'應用於多類分類，而'sigmoid'用於二元分類。

In [44]:
# 解決類別不平均的問題:調類別權重
print(pd.DataFrame(np.array(y_train)).value_counts())

37    1372519
10     611071
0      556203
2      535346
45     424449
36     388560
48     366846
15     339590
16     315655
19     218506
12     208518
39     146285
6      131093
46     125426
26     107350
17     105418
18     102199
34      98428
11      83466
41      76537
25      76053
35      73619
31      71195
22      69172
21      68178
13      62974
20      59353
43      45485
9       40772
33      33679
38      31931
40      22083
23      21683
5       21382
32      18733
3       18694
47      17723
8       15537
30      13267
1       12686
24      11689
7        8745
14       8024
44       7925
42       6825
28       3469
4        2969
27       1302
29         20
dtype: int64


In [45]:
from sklearn.utils import compute_class_weight

train_classes = pd.DataFrame(np.array(y_train))


class_weights = compute_class_weight(
                                        class_weight = "balanced",
                                        classes = np.unique(train_classes),
                                        y = train_classes                                                    
                                    )
class_weights = dict(zip(np.unique(train_classes), class_weights)),
class_weights

/mnt/sdb/home/hongjun/.conda/envs/python38/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


({0: 0.2626640464223402,
  1: 11.51620137255596,
  2: 0.27289739834096993,
  3: 7.8150492464023165,
  4: 49.20664554134217,
  5: 6.8325942667778925,
  6: 1.1144342612667717,
  7: 16.70606410660319,
  8: 9.403007698541861,
  9: 3.5832073631964314,
  10: 0.23907946967250107,
  11: 1.7503478136276436,
  12: 0.7006327061080813,
  13: 2.319918229940053,
  14: 18.207194742303702,
  15: 0.4302085768492738,
  16: 0.46282976861524416,
  17: 1.3858594415777656,
  18: 1.4295103730197447,
  19: 0.6686064941568877,
  20: 2.461451495497193,
  21: 2.14283978134068,
  22: 2.1120472244874358,
  23: 6.737745266441217,
  24: 12.498462709576945,
  25: 1.9209568407853064,
  26: 1.3609178445481593,
  27: 112.2077808081758,
  28: 42.11430689312335,
  29: 7304.7265306122445,
  30: 11.011873868413726,
  31: 2.0520335783727073,
  32: 7.798779192454219,
  33: 4.3378523890924585,
  34: 1.4842781587784462,
  35: 1.984467740831102,
  36: 0.3759896299470993,
  37: 0.1064426289269911,
  38: 4.57531961455153,
  39: 0.

In [46]:
class_weights[0]

{0: 0.2626640464223402,
 1: 11.51620137255596,
 2: 0.27289739834096993,
 3: 7.8150492464023165,
 4: 49.20664554134217,
 5: 6.8325942667778925,
 6: 1.1144342612667717,
 7: 16.70606410660319,
 8: 9.403007698541861,
 9: 3.5832073631964314,
 10: 0.23907946967250107,
 11: 1.7503478136276436,
 12: 0.7006327061080813,
 13: 2.319918229940053,
 14: 18.207194742303702,
 15: 0.4302085768492738,
 16: 0.46282976861524416,
 17: 1.3858594415777656,
 18: 1.4295103730197447,
 19: 0.6686064941568877,
 20: 2.461451495497193,
 21: 2.14283978134068,
 22: 2.1120472244874358,
 23: 6.737745266441217,
 24: 12.498462709576945,
 25: 1.9209568407853064,
 26: 1.3609178445481593,
 27: 112.2077808081758,
 28: 42.11430689312335,
 29: 7304.7265306122445,
 30: 11.011873868413726,
 31: 2.0520335783727073,
 32: 7.798779192454219,
 33: 4.3378523890924585,
 34: 1.4842781587784462,
 35: 1.984467740831102,
 36: 0.3759896299470993,
 37: 0.1064426289269911,
 38: 4.57531961455153,
 39: 0.9986979568120101,
 40: 6.61570124585631,

In [38]:
from keras import backend as K
def weightedLoss(originalLossFunc, weightsList):

    def lossFunc(true, pred):

        axis = -1 #if channels last 
        #axis=  1 #if channels first


        #argmax returns the index of the element with the greatest value
        #done in the class axis, it returns the class index    
        classSelectors = K.argmax(true, axis=axis) 
        classSelectors = tf.cast(classSelectors, tf.int32)
            #if your loss is sparse, use only true as classSelectors

        #considering weights are ordered by class, for each class
        #true(1) if the class index is equal to the weight index   
        classSelectors = [K.equal(i, classSelectors) for i in range(len(weightsList))]

        #casting boolean to float for calculations  
        #each tensor in the list contains 1 where ground true class is equal to its index 
        #if you sum all these, you will get a tensor full of ones. 
        classSelectors = [K.cast(x, K.floatx()) for x in classSelectors]

        #for each of the selections above, multiply their respective weight
        weights = [sel * w for sel,w in zip(classSelectors, weightsList)] 

        #sums all the selections
        #result is a tensor with the respective weight for each element in predictions
        weightMultiplier = weights[0]
        for i in range(1, len(weights)):
            weightMultiplier = weightMultiplier + weights[i]


        #make sure your originalLossFunc only collapses the class axis
        #you need the other axes intact to multiply the weights tensor
        loss = originalLossFunc(true,pred) 
        loss = loss * weightMultiplier

        return loss
    return lossFunc

Using TensorFlow backend.


In [39]:
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1],1)))
model.add(LSTM(units=50))
model.add(Dense(49, activation='softmax'))

# model = Sequential()
# model.add(LSTM(100, input_shape=(X_train1.shape[1],1 ), return_sequences=True))
# model.add(Dropout(0.2))
# model.add(LSTM(200))
# model.add(Dropout(0.2))
# model.add(Dense(49, activation='softmax'))

# model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
model.compile(loss= weightedLoss(tf.keras.losses.sparse_categorical_crossentropy, class_weights[0]), optimizer='adam', metrics = ['accuracy'])
# model.compile(loss= weightedLoss(keras.losses.categorical_crossentropy, weights), optimizer=..., ...)
model.summary()

2021-11-19 01:00:38.794537: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-19 01:00:38.828869: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.545GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 573.69GiB/s
2021-11-19 01:00:38.829678: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-19 01:00:38.832038: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:04:00.0 name: GeForce RTX 2080 Ti computeCapability: 7.5
coreClock: 1.545GHz coreCount: 68 deviceMemorySize: 10.76GiB deviceMemoryBandwidth: 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 21, 50)            10400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense (Dense)                (None, 49)                2499      
Total params: 33,099
Trainable params: 33,099
Non-trainable params: 0
_________________________________________________________________


In [40]:
X_train.shape

(26376068, 21)

In [41]:
y_train.shape

(26376068, 1)

In [42]:
y_train[0:300].shape

(300, 1)

In [43]:
np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))[0:300].shape

(300, 1, 21)

In [44]:
np.reshape(y_train, (y_train.shape[0], 1, y_train.shape[1]))[0:300].shape

(300, 1, 1)

In [45]:
# tf.config.experimental.list_physical_devices('GPU')

In [46]:
# # # from tensorflow.compat.v1 import ConfigProto
# # # from tensorflow.compat.v1 import InteractiveSession
# # # config = ConfigProto()
# # # config.gpu_options.allow_growth = True
# # # session = InteractiveSession(config=config)


# gpu_devices = tf.config.experimental.list_physical_devices('GPU')
# for device in gpu_devices:
#     tf.config.experimental.set_memory_growth(device, True)
    

In [47]:
# from tensorflow.compat.v1 import ConfigProto
# from tensorflow.compat.v1 import InteractiveSession

# config = ConfigProto()
# config.gpu_options.allow_growth = True
# session = InteractiveSession(config=config)

In [48]:
model.fit(X_train1[0:2000000], y_train1[0:2000000], epochs=1, batch_size=1, verbose=1)

2021-11-19 01:00:41.954109: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-11-19 01:00:42.015951: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 3199980000 Hz


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


2021-11-19 01:00:45.519997: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2021-11-19 01:00:48.580054: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7


2000000/2000000 [==============================] - 5371s 3ms/step - loss: 0.0000e+00 - accuracy: 0.0736


In [ ]:
# 試試看跑兩次有沒有比較好
model.fit(X_train1[0:2000000], y_train1[0:2000000], epochs=2, batch_size=1, verbose=1)

In [49]:
# 模型預測
predictions = model.predict(pre_train)

In [50]:
len(predictions)

500000

In [51]:
len(predictions[0])

49

In [52]:
# 按降序獲取前 n 個結果
best_n = np.argsort(-predictions, axis=1)[:, :49]
best_n

array([[45, 47, 42, ...,  2, 18, 23],
       [45, 47, 42, ...,  2, 18, 23],
       [45, 47, 42, ...,  2, 18, 23],
       ...,
       [45, 47, 42, ...,  2, 18, 23],
       [45, 47, 42, ..., 25, 18, 23],
       [45, 42, 47, ...,  2, 18, 23]])

In [53]:
best_n[0]

array([45, 47, 42, 14, 35, 15, 33, 12, 20,  9,  3, 27, 32, 24, 29, 46, 39,
        6, 16, 21, 43, 41, 13,  1, 36, 37, 40, 28,  5, 22,  0,  4,  7, 26,
       17, 38, 48, 44, 10, 34, 31,  8, 30, 11, 25, 19,  2, 18, 23])

In [54]:
# 建立best_predictions_final來儲存前三高的預測值
best_predictions_final = np.zeros((len(best_n), 3), dtype=int)

In [55]:
best_predictions_final

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       ...,
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]])

In [56]:
# 過濾16個前三高的標籤出來
for a in range(len(best_n)):
    sum = 0
    for i in range(49): 
        if (best_n[a, i] == [2,6,10,12,13,15,18,19,21,22,25,26,36,37,39,48]).any():
            best_predictions_final[a, sum] = int(best_n[a, i])
            sum += 1 
        if sum == 3: 
            break

In [57]:
submission = pd.DataFrame({ 'chid':predict.chid,'top1':best_predictions_final[0:len(best_predictions_final),0] , 'top2':best_predictions_final[0:len(best_predictions_final),1] , 'top3':best_predictions_final[0:len(best_predictions_final),2] })

In [58]:
filename = 'submission-200W-砍欄位-不留chid.csv'
submission.to_csv(filename,index=False)
print(submission)
print('Saved file: ' + filename)

            chid  top1  top2  top3
0       10128239    15    12    39
1       10077943    15    12    39
2       10277876    15    12    39
3       10364842    15     6    12
4       10392717    15    12    39
...          ...   ...   ...   ...
499995  10433239    15    12    39
499996  10235119    15    12    39
499997  10267595    15    12    39
499998  10160606    15     6    12
499999  10141759    15     6    39

[500000 rows x 4 columns]
Saved file: submission-200W-砍欄位-不留chid.csv


In [59]:
# 保存模型
model.save('LSTM_model_200W-砍欄位-不留chid.h5') 